In [1]:
from top2vec import Top2Vec

model_file = "/home/cds/Documents/Models/top2vec.model"

model = Top2Vec.load(model_file)

In [2]:
def topic_parser(file):
    topics = dict()
    with open(file, 'r') as fp:
        for line in fp.readlines():
            elems = line.split('|')
            topics[int(elems[0])] = elems[1]
    return topics

In [3]:
topics_file = "/home/cds/Documents/Topics/topics_full.txt"

topics = topic_parser(topics_file)

In [31]:
keywords = topics[51].split(" ")

keywords.remove("document")
keywords.remove("mention")
keywords.remove("discuss")
print(keywords)

['government', 'assistance', 'airbus', 'industrie', 'trade', 'dispute', 'airbus', 'aircraft', 'producer', 'issue', 'subsidy']


In [32]:
document_scores, document_ids = model.search_documents_by_keywords(keywords=keywords, num_docs=10000)
#for score, doc_id in zip(document_scores, document_ids):
#    print(f"Document: {doc_id}, Score: {score}")
  #  print("-----------")
 #   print()

In [4]:
def save_runs(doc_ids, doc_scores, topic_no, file):
    with open(file,  'a') as fp:
        for i, (doc_id, doc_score) in enumerate(zip(doc_ids, doc_scores)):
            fp.write(str(topic_no) + " Q0 " + doc_id + " " + str(i) + " " + str(doc_score) + " test" + "\n")


In [33]:
run_file = "/home/cds/Documents/Runs/run_desc.txt"

save_runs(document_ids, document_scores, 51, run_file)

In [12]:
trcv_manifest = {"trcv1":["wsj87", "wsj88", "wsj89", "fr89", "ap89", "doe", "zf1"],
                 "trcv2":["wsj90", "wsj91", "wsj92", "fr88", "ap88", "zf2"],
                 "trcv3":["sjmn", "ap90", "pt", "zf3"],
                 "trcv4":["ft", "cr", "fr94"],
                 "trcv5":["fbis", "la"],
                 "trcv4_nocr":["ft", "fr94"]}

trec_manifest = {"trec1":["trcv1", "trcv2"],
                 "trec2":["trcv1", "trcv2"],
                 "trec3":["trcv1", "trcv2"],
                 "trec4":["trcv2", "trcv3"],
                 "trec5":["trcv2", "trcv4"],
                 "trec6":["trcv4", "trcv5"],
                 "trec7":["trcv4_nocr", "trcv5"],
                 "trec8":["trcv4_nocr", "trcv5"]}

def get_doc_prefix(trec_no):
    prefix_list = set()
    for trcv in trec_manifest["trec"+str(trec_no)]:
        for prefix in trcv_manifest[trcv]:
            prefix_list.add(prefix)
    return prefix_list

def result_filter(doc_scores, doc_ids, trec_no):
    filtered_scores = list()
    filtered_ids = list()

    prefixes = get_doc_prefix(trec_no)

    for i, doc_id in enumerate(doc_ids):
        for prefix in prefixes:
            if doc_id.lower().startswith(prefix):
                filtered_ids.append(doc_id)
                filtered_scores.append(doc_scores[i])

    return filtered_scores, filtered_ids

def get_topic_list(trec_no):
    return [i for i in range(50*trec_no+1, 50*(trec_no+1)+1)]

def check_keywords(keywords):
    oov_keywords = list()
    filtered_keywords = list()
    vocab = model.model.wv.vocab
    for word in keywords:
        if word not in vocab:
            oov_keywords.append(word)
        else:
            filtered_keywords.append(word)
    return filtered_keywords, oov_keywords

In [13]:
trec_no = 5

run_file = "/home/cds/Documents/Runs/run_baseline.txt"

oov = list()

for topic_no in get_topic_list(trec_no):
    keywords = topics[topic_no].split(" ")
    filtered_keywords, oov_keywords = check_keywords(keywords)
    oov.extend(oov_keywords)
    document_scores, document_ids = model.search_documents_by_keywords(keywords=filtered_keywords, num_docs=10000)
    filtered_scores, filtered_ids = result_filter(document_scores, document_ids, trec_no)
    save_runs(filtered_ids, filtered_scores, topic_no, run_file)

In [9]:
def search_documents_by_topkeys(model=model, keywords=keywords, num_docs=num_docs):
    _,_,_,topics_nums = model.search_topics(keywords, 10)

    for topics_no in topics_nums:
        model.get_documents_by_topic()
        model.search_documents_by_topic(topics_no, model.get, return_documents=False)



In [1]:
test = ["apple", "sads"]
t = ["apple"]
model._get_word_vectors(t)

NameError: name 'model' is not defined

In [2]:
ds = model._get_document_vectors()
vs = model.model.wv.vectors

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

